In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [38]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [39]:
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory
from google.adk.plugins.logging_plugin import LoggingPlugin
import json
import logging
import os
print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [40]:
# Clean up any previous logs
if os.path.exists("logger.log"):
    os.remove("logger.log")
    print("🧹 Cleaned up logger.log")

# Configure logging with DEBUG log level
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)

In [41]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [42]:
from google.adk.runners import Runner
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
    MODEL_NAME = "gemini-2.5-flash-lite",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [43]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers

# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]['base_url']

    try:
        path_parts = baseURL.split('/')
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix

print("✅ web interface defined.")

✅ web interface defined.


In [44]:
import json
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import DatabaseSessionService

APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session
MODEL_NAME = "gemini-2.5-flash-lite"



def find_scholarships(profile: dict, dataset_path: str, top_k: int = 5):
    import json

    try:
        with open(dataset_path, "r") as f:
            scholarships = json.load(f)
    except Exception as e:
        return {"status": "error", "error_message": f"Failed to load dataset: {str(e)}"}

    eligible = []

    funding_pref = str(profile.get("funding", "")).lower().replace("$", "").replace(",", "")

    for s in scholarships:
        degree_ok = (
            str(profile.get("degree", "")).lower() == "any"
            or profile.get("degree") in s.get("degrees", "")
        )

        country_ok = (
            str(profile.get("country", "")).lower() == "any"
            or profile.get("country", "").lower() in s.get("location", "").lower()
        )

        if degree_ok and country_ok:
            score = 0

            scholarship_funds_str = str(s.get("funds", "")).lower().replace("$", "").replace(",", "")

            if funding_pref == "any" or funding_pref in scholarship_funds_str:
                score += 1

            s["score"] = score
            eligible.append(s)

    if not eligible:
        return {"status": "success", "scholarships": [], "message": "No matching scholarships found"}

    return {
        "status": "success",
        "scholarships": sorted(eligible, key=lambda x: x["score"], reverse=True)[:top_k]
    }

        
          
# Define scope levels for state keys (following best practices)
USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")


# This demonstrates how tools can write to session state using tool_context.
# The 'user:' prefix indicates this is user-specific data.
def save_userinfo(
    tool_context: ToolContext, **kwargs
) -> dict[str, any]:
    """
    Flexible tool to save ANY user information into session state.
    All keys are auto-prefixed with 'user:'.

    Example usage:
    save_userinfo(name="Imon", country="Bangladesh", degree="BSc CSE")

    Stored as:
        user:name = "Imon"
        user:country = "Bangladesh"
        user:degree = "BSc CSE"
    """

    # Store all passed keyword arguments
    for key, value in kwargs.items():
        namespaced_key = f"user:{key}"
        tool_context.state[namespaced_key] = value

    return {"status": "success", "saved_fields": list(kwargs.keys())}


def retrieve_userinfo(tool_context: ToolContext) -> dict[str, any]:
    """
    Flexible tool to retrieve ALL user-related info 
    (keys starting with 'user:') from session state.
    Returns a clean dictionary without the prefix.
    """

    user_data = {}

    # Access the State object's combined dictionary safely
    current_state = {**tool_context.state._value, **tool_context.state._delta}

    for key, value in current_state.items():
        if key.startswith("user:"):
            clean_key = key.replace("user:", "")
            user_data[clean_key] = value

    return {"status": "success", "data": user_data}

print("✅ Tools created.")



# Wrapper functions for Dataset
dataset_path = "/kaggle/input/scholarships-clean/scholarships_clean.json"

def agent_scholarship_finder(profile: dict, top_k: int = 5) -> dict:
    """Finds scholarships for a user profile using a predefined dataset."""
    return find_scholarships(profile=profile, dataset_path=dataset_path, top_k=top_k)



✅ Tools created.


InMemoryMemoryService - Built-in service for prototyping and testing (keyword matching, no persistence)
session_service → Manages conversation threads and events
memory_service → Provides long-term knowledge storage

using-load_memory (Reactive)
where -Agent decides when to search memory
Only retrieves when the agent thinks it's needed
More efficient (saves tokens)
Risk: Agent might forget to search

In [45]:
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Use google_search tool to find scholarships worldwide.
    Return raw search results.
    """,
    tools=[google_search]
)


In [46]:

scholarship_agent = LlmAgent(
    name="scholarship_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a smart scholarship recommendation assistant.

--- Memory Rules ---
1. Whenever the user explicitly provides their name or country, you MUST call the `save_userinfo` tool to store it in session state using the `user:` prefix.
2. Before giving any scholarship recommendations, ALWAYS call the `retrieve_userinfo` tool to check whether user information is already stored.
3. If the user's name or country is missing after retrieval, politely ask the user for the missing details.

--- Scholarship Search Rules ---
4. When the user provides a scholarship query (degree, preferred country, funding preference), first call the `agent_scholarship_finder` tool.
5. If the tool returns an empty list, you MUST call the `google_search_scholarships` tool as a fallback to find external scholarship information.
6. If both searches return no results, politely inform the user.

--- Output Formatting Rules ---
7. Format the final scholarship results as a bulleted list.
8. For EACH scholarship, use EXACTLY this template:

   **[Title]**
   * **Degrees**: [List of degrees]
   * **Funds**: [Funding amount]
   * **Country**: [Country]
   * **Deadline**: [Deadline]

9. If any information is missing, write "Not specified".
10. After the list, briefly explain WHY these scholarships match the user’s profile.""",
    tools=[find_scholarships,save_userinfo, retrieve_userinfo, AgentTool(agent=google_search_agent)] 
)

print("✅ Scholarship Finder Agent created with structured instructions.")
print("🔧 Available tool: agent_scholarship_finder")


# Define the local database URL
db_url = "sqlite:///scholarship_agent_data.db" 

# Initialize the DatabaseSessionService
session_service = DatabaseSessionService(db_url=db_url)

# --- 3. Step 2: Initialize the Persistent Runner ---
memory_service = (
    InMemoryMemoryService()
)  # ADK's built-in Memory Service for development and testing

# Use the base Runner class and pass the session_service argument
scholarship_runner = Runner(
    agent=scholarship_agent, 
    app_name=APP_NAME, 
    session_service=session_service,
    memory_service=memory_service,
    plugins=[LoggingPlugin()]
)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: scholarship_agent_data.db")
print(f"   - Sessions will survive restarts!")

await run_session(
    scholarship_runner,
    [
        "Hi, I am Sam! ",
        "I want to study in Any Country and I am looking for Any Funding Amount",  # This time, the agent should remember!
    ],
    "stateful-agentic-session",
)


✅ Scholarship Finder Agent created with structured instructions.
🔧 Available tool: agent_scholarship_finder
✅ Upgraded to persistent sessions!
   - Database: scholarship_agent_data.db
   - Sessions will survive restarts!

 ### Session: stateful-agentic-session

User > Hi, I am Sam! 
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-b4190c31-12b1-42b6-a0ee-2c0d34236c0c
[logging_plugin]    Session ID: stateful-agentic-session
[logging_plugin]    User ID: default
[logging_plugin]    App Name: default
[logging_plugin]    Root Agent: scholarship_agent
[logging_plugin]    User Content: text: 'Hi, I am Sam!'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-b4190c31-12b1-42b6-a0ee-2c0d34236c0c
[logging_plugin]    Starting Agent: scholarship_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: scholarship_agent
[logging_plugin]    Invocation ID: e-b4190c31-12b1-42b6-a0ee-2c0d34236c0c
[logging_plugin] 🧠 LLM REQUEST
[loggin

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: scholarship_agent
[logging_plugin]    Content: text: 'Hello Sam! It's nice to meet you.' | function_call: save_userinfo
[logging_plugin]    Token Usage - Input: 602, Output: 27
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 8eea1915-45d8-4c01-bf90-aa3bfa7d8d37
[logging_plugin]    Author: scholarship_agent
[logging_plugin]    Content: text: 'Hello Sam! It's nice to meet you.' | function_call: save_userinfo
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['save_userinfo']
gemini-2.5-flash-lite >  Hello Sam! It's nice to meet you.
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: save_userinfo
[logging_plugin]    Agent: scholarship_agent
[logging_plugin]    Function Call ID: adk-47020213-8f9b-42e5-9ff6-ee97f4df5457
[logging_plugin]    Arguments: {'name': 'Sam'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: save_userinfo
[logging_plugin]    Agent: schola

In [47]:
#checking agent is stateful or not
await run_session(
    scholarship_runner,
    ["What did I ask you about earlier?", "And remind me, what's my name?"],
    "stateful-agentic-session",











    
)


 ### Session: stateful-agentic-session

User > What did I ask you about earlier?
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-e1796d6d-cee6-43aa-ad02-ddf569f31c16
[logging_plugin]    Session ID: stateful-agentic-session
[logging_plugin]    User ID: default
[logging_plugin]    App Name: default
[logging_plugin]    Root Agent: scholarship_agent
[logging_plugin]    User Content: text: 'What did I ask you about earlier?'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-e1796d6d-cee6-43aa-ad02-ddf569f31c16
[logging_plugin]    Starting Agent: scholarship_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: scholarship_agent
[logging_plugin]    Invocation ID: e-e1796d6d-cee6-43aa-ad02-ddf569f31c16
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: scholarship_agent
[logging_plugin]    System Instruction: '
    You are a smart scholarship recommendation assist

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: scholarship_agent
[logging_plugin]    Content: function_call: retrieve_userinfo
[logging_plugin]    Token Usage - Input: 766, Output: 11
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 6e4f51e3-bdee-4edc-8c37-026d06d1b263
[logging_plugin]    Author: scholarship_agent
[logging_plugin]    Content: function_call: retrieve_userinfo
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['retrieve_userinfo']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: retrieve_userinfo
[logging_plugin]    Agent: scholarship_agent
[logging_plugin]    Function Call ID: adk-aeb53e42-6f27-418e-a147-a8dc4c2e6d53
[logging_plugin]    Arguments: {}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: retrieve_userinfo
[logging_plugin]    Agent: scholarship_agent
[logging_plugin]    Function Call ID: adk-aeb53e42-6f27-418e-a147-a8dc4c2e6d53
[logging_plugin]    Result: {'status': 'success',

In [48]:
await run_session(
    scholarship_runner,
    ["What is the capital of India?", "Hello! What is my name?"],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > What is the capital of India?
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-0f20cc10-7144-4a13-82bb-ff6e17de6290
[logging_plugin]    Session ID: test-db-session-01
[logging_plugin]    User ID: default
[logging_plugin]    App Name: default
[logging_plugin]    Root Agent: scholarship_agent
[logging_plugin]    User Content: text: 'What is the capital of India?'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-0f20cc10-7144-4a13-82bb-ff6e17de6290
[logging_plugin]    Starting Agent: scholarship_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: scholarship_agent
[logging_plugin]    Invocation ID: e-0f20cc10-7144-4a13-82bb-ff6e17de6290
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: scholarship_agent
[logging_plugin]    System Instruction: '
    You are a smart scholarship recommendation assistant.

--- Memory Rul

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: scholarship_agent
[logging_plugin]    Content: function_call: retrieve_userinfo
[logging_plugin]    Token Usage - Input: 602, Output: 11
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: a3b4f244-993d-4606-b111-e6e740cfb492
[logging_plugin]    Author: scholarship_agent
[logging_plugin]    Content: function_call: retrieve_userinfo
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['retrieve_userinfo']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: retrieve_userinfo
[logging_plugin]    Agent: scholarship_agent
[logging_plugin]    Function Call ID: adk-8a1d4f34-995c-4f62-968d-b0db3dda7794
[logging_plugin]    Arguments: {}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: retrieve_userinfo
[logging_plugin]    Agent: scholarship_agent
[logging_plugin]    Function Call ID: adk-8a1d4f34-995c-4f62-968d-b0db3dda7794
[logging_plugin]    Result: {'status': 'success',

here i.e., two sessions do not share information


seeing whats in the database


In [49]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("scholarship_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

['app_name', 'session_id', 'author', 'content']
('default', 'stateful-agentic-session', 'user', '{"parts": [{"text": "Hi, I am Sam! "}], "role": "user"}')
('default', 'stateful-agentic-session', 'scholarship_agent', '{"parts": [{"text": "Hello Sam! It\'s nice to meet you."}, {"function_call": {"id": "adk-47020213-8f9b-42e5-9ff6-ee97f4df5457", "args": {"name": "Sam"}, "name": "save_userinfo"}}], "role": "model"}')
('default', 'stateful-agentic-session', 'scholarship_agent', '{"parts": [{"function_response": {"id": "adk-47020213-8f9b-42e5-9ff6-ee97f4df5457", "name": "save_userinfo", "response": {"status": "success", "saved_fields": []}}}], "role": "user"}')
('default', 'stateful-agentic-session', 'scholarship_agent', '{"parts": [{"text": "I have saved your name. Could you please tell me which country you are interested in applying from, and what is your highest degree?"}], "role": "model"}')
('default', 'stateful-agentic-session', 'user', '{"parts": [{"text": "I want to study in Any Coun

In [50]:
 
#events can become very large, leading to slower performance and higher costs.
from google.adk.apps.app import App, EventsCompactionConfig,ResumabilityConfig

scholarship_app_compacting = App(
    name="scholarship_app_compacting",  # Use the same app name as before
    root_agent=scholarship_agent,
    resumability_config=ResumabilityConfig(is_resumable=False),
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1, 

    ),
)

db_url = "sqlite:///scholarship_agent_data.db"
session_service = DatabaseSessionService(db_url=db_url)

# 3. Create the new Runner using the compacting App
scholarship_runner_compacting = Runner(
    app=scholarship_app_compacting, 
    session_service=session_service
)

print("✅ Scholarship App redefined with Events Compaction!")
print(f"🔧 Compaction Interval: {scholarship_app_compacting.events_compaction_config.compaction_interval} turns")
print("🚀 Use 'scholarship_runner_compacting' for optimized, persistent sessions.")

✅ Scholarship App redefined with Events Compaction!
🔧 Compaction Interval: 3 turns
🚀 Use 'scholarship_runner_compacting' for optimized, persistent sessions.


/tmp/ipykernel_48/1535822819.py:7: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=False),
/tmp/ipykernel_48/1535822819.py:8: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(




# 📝 Issue & Solution Note: *Resumability + Compaction Error*

## ❗ Issue

When using **Google ADK agents**, enabling **`resumability=True`** and **`compaction=True`** at the same time causes session metadata corruption.
This results in runtime errors such as:

```
'dict' object has no attribute 'start_timestamp'
```

### **Why it happens**

* `resumability` stores metadata (e.g., timestamps, history pointers).
* `compaction` rewrites/compacts session data.
* These two mechanisms **conflict internally**, leading to malformed metadata stored in the database.

---

## ✅ Solution

### **1. Disable resumability when compaction is used**

```python
session_service = DatabaseSessionService(
    database=...,
    resumable=False
)
```

### **2. Delete the old database/session**

If the data is already corrupted, you must **remove/reset** the previous session storage (local DB or Firestore).

### **3. Re-run the agent**

After resetting the DB and disabling `resumability`, the agent will run correctly without errors.

---

## ✔ Final Recommendation

> **Never use `resumability=True` and `compaction=True` together.**
> For projects using compaction, always set `resumable=False`.

In [51]:
#turn1
await run_session(
    scholarship_runner_compacting,
    "my name is imon from Bangladesh.I am a Masters student in the USA. Find me scholarships with $1000 funding .",
    "Compaction_demo",
)
# --- Turn 2:
await run_session(
    scholarship_runner_compacting,
    "Based on those results, can you tell me the deadline for the first scholarship you found?",
    "Compaction_demo",
)
# --- Turn 3:
await run_session(
    scholarship_runner_compacting,
    "Which of those scholarships is also open to Bachelor students?",
    "Compaction_demo",
)
#turn 4
await run_session(
    scholarship_runner_compacting,
    "What country did I say I was interested in for my studies?",
    "Compaction_demo",
)


 ### Session: Compaction_demo

User > my name is imon from Bangladesh.I am a Masters student in the USA. Find me scholarships with $1000 funding .


gemini-2.5-flash-lite >  I found a few scholarships that might be a good fit for you.

**Mpower Financing Scholarships**
*   **Degrees**: Master's
*   **Funds**: Up to US$1,000 (monthly scholarships)
*   **Country**: USA
*   **Deadline**: Not specified

**Mpower Financing Women in STEM Scholarships**
*   **Degrees**: Master's
*   **Funds**: Up to US$5,000
*   **Country**: USA
*   **Deadline**: Not specified

**Mpower Financing Global Citizen Scholarships**
*   **Degrees**: Master's
*   **Funds**: Up to US$8,000
*   **Country**: USA
*   **Deadline**: Not specified

**General Fully Funded Master's Scholarships in the USA**
*   **Degrees**: Master's
*   **Funds**: Approximately US$1000 monthly stipend (plus tuition, accommodation, health insurance, and travel allowance)
*   **Country**: USA
*   **Deadline**: Not specified

These scholarships match your profile as a Master's student from Bangladesh seeking funding in the USA. The Mpower Financing scholarships offer specific amounts that al

In [52]:
# Get the final session state
final_session = await session_service.get_session(
    app_name=scholarship_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="Compaction_demo",
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("\n✅ SUCCESS! Found the Compaction Event:")
        print(f"  Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n❌ No compaction event found. Try increasing the number of turns in the demo."
    )

--- Searching for Compaction Summary Event ---

❌ No compaction event found. Try increasing the number of turns in the demo.


**Here we create Cv/sop agent with HITL pause and resume as we see it in the 
course materials 2(b) ,**

In [53]:
import uuid
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.function_tool import FunctionTool

# 1. Define the Review Tool 
def submit_draft_for_review(
    draft_text: str, document_type: str, tool_context: ToolContext
) -> dict:
    """
    Submits a draft (SOP or CV) for human review.
    This tool PAUSES execution until the human approves or rejects.
    """
    
    # SCENARIO 1: First call - PAUSE for Human Review
    if not tool_context.tool_confirmation:
        
        # We attach the FULL DRAFT to the confirmation payload so the human can see it
        tool_context.request_confirmation(
            hint=f"Review {document_type} Draft",
            payload={
                "document_type": document_type,
                "draft_text": draft_text 
            },
        )
        
        return { 
            "status": "pending_review",
            "message": f"Draft submitted. Waiting for human feedback on: {document_type}"
        }

    # SCENARIO 2: Resuming after Human Decision
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "message": "The human APPROVED the draft. You can now finalize formatting and output the result."
        }
    else:
        return {
            "status": "rejected",
            "message": "The human REJECTED the draft. Please ask the user for specific feedback on what to change."
        }


print("✅ Review Tool created.")

✅ Review Tool created.


In [54]:
refiner_agent = LlmAgent(
    name="refiner_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
    You are an academic editor.
    Your job is to take raw text and return:
    - grammatically correct
    - more concise
    - academic tone
    - logically structured
    Do NOT add new ideas. Only improve writing.
    """,
)


In [55]:
# 1️⃣ Redefine SOP Agent as TEXT ONLY (No change needed, this is robust)
sop_agent = LlmAgent(
    name="sop_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
    You are an expert academic SOP writer.
    1. Only generate a professional SOP based on the user profile and scholarship details.
    2. Do NOT call any tools.
    3. Do NOT save anything. Just output the text of the SOP.
    """
    
)

# 2️⃣ Redefine CV Agent as TEXT ONLY (No change needed, this is robust)
cv_agent = LlmAgent(
    name="cv_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
    You are an expert CV writer.
    1. Only generate a professional CV based on the user profile and scholarship details.
    2. Do NOT call any tools.
    3. Do NOT save anything. Just output the text of the CV.
    """
    
)

print("✅ SOP and CV Agents")


✅ SOP and CV Agents


In [56]:
from google.adk.apps.app import App, ResumabilityConfig

# 3️⃣ Orchestrator Agent: handles all tool calls including HITL
orchestrator_agent = LlmAgent(
    name="orchestrator_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Orchestrator for the Scholarship System.
    
    1. At the start of a new session, ALWAYS call `retrieve_userinfo` to check for saved user data.
    2. If the user asks to find scholarships, call the `scholarship_agent`.
    
    3. **Document Generation Pipeline (SOP/CV):** When the user asks to generate a CV or SOP, you MUST follow this strict sequence:
        a. **Generate Draft:** Call the appropriate sub-agent (`cv_agent` or `sop_agent`) to generate the draft text.
        b. **Refine Draft (AITL):** Immediately call the `refiner_agent` tool, passing the generated draft text as input. The refiner will return the polished draft.
        c. **DISPLAY DRAFT TO USER:** Immediately output the polished draft text (the result of the refiner_agent) to the user.
        d. **Submit for HITL:** Then, immediately call the `submit_draft_for_review` tool, passing the SAME polished draft text to the tool.
        e. Wait for the tool's status (approval or rejection) and inform the user of the outcome, asking them to confirm the submission.
    4. Once the draft is approved by the human, call `save_userinfo` to store the final output (e.g., user:last_sop or user:last_cv).
    5. Always return concise, clear, and action-oriented responses to the user.
    """,
    tools=[
        save_userinfo,
        retrieve_userinfo,
        # API FIX: Removed 'name' argument from AgentTool
        AgentTool(agent=scholarship_agent), 
        AgentTool(agent=sop_agent),
        AgentTool(agent=cv_agent),
        FunctionTool(func=submit_draft_for_review),
        AgentTool(agent=refiner_agent),
        AgentTool(agent=google_search_agent),
    ],
    
)

# 4️⃣ Set up database-backed session (No change needed)
db_url = "sqlite:///scholarship_orchestrator.db"
session_service = DatabaseSessionService(db_url=db_url)

# 5️⃣ Wrap Orchestrator in a resumable App (Assuming necessary imports are present)
orchestrator_app = App(
    name="scholarship_orchestrator_app",
    root_agent=orchestrator_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
    plugins=[LoggingPlugin()]
)


print("✅ Orchestrator App ready with robust configuration.")

✅ Orchestrator App ready with robust configuration.


/tmp/ipykernel_48/2932023836.py:44: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),




# 📝 Why `resumability=True` Is Used (And Why It’s Safe Here)

## ✅ Reason for Using `resumability=True`

In the **Orchestrator Agent**, `resumability=True` is intentionally enabled because:

### ✔ The Orchestrator needs long-term memory

It must **remember previous user data**, such as:

* last generated CV/SOP
* stored scholarship preferences
* previous HITL review status
* user profile information

This memory is required for:

* calling `retrieve_userinfo` at the start of each new session
* tracking document creation workflows
* maintaining continuity between conversations

So **resumability is essential** for the Orchestrator.

---

## 🚫 Why This Does NOT cause errors

You **did NOT enable Events Compaction** in this setup.

### ❗ Important Rule

> **Resumability and Compaction cannot be used together.**

Since you **only enabled resumability**, and **no compaction**, the session metadata is stored correctly.
Therefore, no corruption occurs, and no `start_timestamp` error will appear.


In [57]:

# Initialize the Runner using the 'app' parameter
orchestrator_runner = Runner(
    app=orchestrator_app,            
    session_service=session_service, 
    memory_service=memory_service,  
)

print("✅ Orchestrator Runner initialized successfully!")

✅ Orchestrator Runner initialized successfully!


In [58]:
# Ensure your helper function 'run_session' is defined (from your previous cells)
# If not, paste the helper function definition here first.

await run_session(
    orchestrator_runner,
    [
        "Hi, I found a scholarship!", 
        "Can you help me write a Statement of Purpose (SOP) for it?"
    ],
    "orchestrator-test-session-01"
)


 ### Session: orchestrator-test-session-01

User > Hi, I found a scholarship!
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-21a12a2a-3885-403a-b5ef-642dd750221a
[logging_plugin]    Session ID: orchestrator-test-session-01
[logging_plugin]    User ID: default
[logging_plugin]    App Name: scholarship_orchestrator_app
[logging_plugin]    Root Agent: orchestrator_agent
[logging_plugin]    User Content: text: 'Hi, I found a scholarship!'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-21a12a2a-3885-403a-b5ef-642dd750221a
[logging_plugin]    Starting Agent: orchestrator_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: orchestrator_agent
[logging_plugin]    Invocation ID: e-21a12a2a-3885-403a-b5ef-642dd750221a
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: orchestrator_agent
[logging_plugin]    System Instruction: '
    You are the Orchestrator for t

In [59]:
await run_session(
    orchestrator_runner,
    [
        "I am applying for a Master's in Data Science.",
        "The university is Stanford."
    ],
    "orchestrator-test-session-01"  # MUST be the same ID to maintain context
)


 ### Session: orchestrator-test-session-01

User > I am applying for a Master's in Data Science.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-f7203207-4a8d-4550-a598-7f9f482aa22f
[logging_plugin]    Session ID: orchestrator-test-session-01
[logging_plugin]    User ID: default
[logging_plugin]    App Name: scholarship_orchestrator_app
[logging_plugin]    Root Agent: orchestrator_agent
[logging_plugin]    User Content: text: 'I am applying for a Master's in Data Science.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-f7203207-4a8d-4550-a598-7f9f482aa22f
[logging_plugin]    Starting Agent: orchestrator_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: orchestrator_agent
[logging_plugin]    Invocation ID: e-f7203207-4a8d-4550-a598-7f9f482aa22f
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: orchestrator_agent
[logging_plugin]    System Instruction

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: orchestrator_agent
[logging_plugin]    Content: function_call: sop_agent
[logging_plugin]    Token Usage - Input: 2250, Output: 25
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 161f206c-5a40-4eaf-875a-3f9975661528
[logging_plugin]    Author: orchestrator_agent
[logging_plugin]    Content: function_call: sop_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['sop_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: sop_agent
[logging_plugin]    Agent: orchestrator_agent
[logging_plugin]    Function Call ID: adk-9509bbeb-572e-4b3b-a9eb-1fa2d5a5effd
[logging_plugin]    Arguments: {'request': "I am applying for a Master's in Data Science."}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-37cd36c7-87ee-4a17-8c24-969bb07ee66f
[logging_plugin]    Session ID: f601efff-bbf5-4bc4-93f4-2c7c71d77693
[logging_plugin]    User ID: default
[logg

In [60]:
# Continue the session to see the result of the internal tool chain (Generate -> Refine)
await run_session(
    orchestrator_runner,
    ["Continue."], # A neutral prompt to trigger the next execution cycle
    "orchestrator-test-session-04" 
)


 ### Session: orchestrator-test-session-04

User > Continue.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-e1d04e99-c131-439f-8859-eb4a5be91d5f
[logging_plugin]    Session ID: orchestrator-test-session-04
[logging_plugin]    User ID: default
[logging_plugin]    App Name: scholarship_orchestrator_app
[logging_plugin]    Root Agent: orchestrator_agent
[logging_plugin]    User Content: text: 'Continue.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-e1d04e99-c131-439f-8859-eb4a5be91d5f
[logging_plugin]    Starting Agent: orchestrator_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: orchestrator_agent
[logging_plugin]    Invocation ID: e-e1d04e99-c131-439f-8859-eb4a5be91d5f
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: orchestrator_agent
[logging_plugin]    System Instruction: '
    You are the Orchestrator for the Scholarship System.
    
    1.

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: orchestrator_agent
[logging_plugin]    Content: function_call: refiner_agent
[logging_plugin]    Token Usage - Input: 5622, Output: 868
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 39dc00e8-d0a1-421d-8981-11fe3b9f8e58
[logging_plugin]    Author: orchestrator_agent
[logging_plugin]    Content: function_call: refiner_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['refiner_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: refiner_agent
[logging_plugin]    Agent: orchestrator_agent
[logging_plugin]    Function Call ID: adk-67ed10b9-1da3-4baa-9beb-151e35853c97
[logging_plugin]    Arguments: {'request': '**Statement of Purpose**\n\nDear Scholarship Selection Committee,\n\nI am writing to express my profound interest in the [Scholarship Name] for the Master\'s program in Artificial Intelligence at [University Name]. My undergraduate studies in Electrical Eng

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: orchestrator_agent
[logging_plugin]    Content: text: 'I have refined the Statement of Purpose draft. Please review the updated version:

**Refined Statement of Purpose Draft:**
**Statement of Purpose**

Dear Scholarship Selection Committee,

I am writing...' | function_call: submit_draft_for_review
[logging_plugin]    Token Usage - Input: 7304, Output: 1664
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 5b27abe2-7a33-43e9-b518-10455df6ece4
[logging_plugin]    Author: orchestrator_agent
[logging_plugin]    Content: text: 'I have refined the Statement of Purpose draft. Please review the updated version:

**Refined Statement of Purpose Draft:**
**Statement of Purpose**

Dear Scholarship Selection Committee,

I am writing...' | function_call: submit_draft_for_review
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['submit_draft_for_review']
gemini-2.5-flash-lite >  I have refined the S

/usr/local/lib/python3.11/dist-packages/google/adk/tools/tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(


In [61]:
await run_session(
    orchestrator_runner,
    [
        "Yes, I approve. Please submit this draft immediately for human review."
    ],
    "orchestrator-test-session-04" # Continue the session
)


 ### Session: orchestrator-test-session-04

User > Yes, I approve. Please submit this draft immediately for human review.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-5625ea7d-e3e9-4de4-809f-c1e5c51036a2
[logging_plugin]    Session ID: orchestrator-test-session-04
[logging_plugin]    User ID: default
[logging_plugin]    App Name: scholarship_orchestrator_app
[logging_plugin]    Root Agent: orchestrator_agent
[logging_plugin]    User Content: text: 'Yes, I approve. Please submit this draft immediately for human review.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-5625ea7d-e3e9-4de4-809f-c1e5c51036a2
[logging_plugin]    Starting Agent: orchestrator_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: orchestrator_agent
[logging_plugin]    Invocation ID: e-5625ea7d-e3e9-4de4-809f-c1e5c51036a2
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: orchest

In [ ]:
# The Orchestrator is waiting for the result of the tool call.
# This simulates the tool returning a successful "Approved" status.
await run_session(
    orchestrator_runner,
    [
        "The human reviewer has completed the review. The document status is: Approved."
    ],
    "orchestrator-test-session-04" # Continue the session
)


 ### Session: orchestrator-test-session-04

User > The human reviewer has completed the review. The document status is: Approved.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-50d8bad4-f2f0-4654-91d9-4f4a25443471
[logging_plugin]    Session ID: orchestrator-test-session-04
[logging_plugin]    User ID: default
[logging_plugin]    App Name: scholarship_orchestrator_app
[logging_plugin]    Root Agent: orchestrator_agent
[logging_plugin]    User Content: text: 'The human reviewer has completed the review. The document status is: Approved.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-50d8bad4-f2f0-4654-91d9-4f4a25443471
[logging_plugin]    Starting Agent: orchestrator_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: orchestrator_agent
[logging_plugin]    Invocation ID: e-50d8bad4-f2f0-4654-91d9-4f4a25443471
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]  

In [ ]:
await run_session(
    orchestrator_runner,
    ["What is the final status of my Statement of Purpose, and what degree was it for?"],
    "orchestrator-test-session-04" 
)

In [ ]:
await run_session(
    orchestrator_runner,
    ["You previously confirmed the SOP was approved and saved. Please retrieve the saved SOP text for the Master's in Data Science."],
    "orchestrator-test-session-04" 
)

In [ ]:
# Force the agent to fulfill the request by prioritizing the text retrieval over the apology/limitation
await run_session(
    orchestrator_runner,
    ["Since you said you have a copy of the approved SOP, please provide it now."],
    "orchestrator-test-session-04" 
)

In [ ]:
# Clean up any existing database to start fresh (if Notebook is restarted)
import os

if os.path.exists("scholarship_agent_data.db"):
    os.remove("scholarship_agent_data.db")
print("✅ Cleaned up old database files")

In [ ]:
await run_session(
    orchestrator_runner,
    ["Generate a detailed Statement of Purpose and a structured CV draft for a scholarship in Artificial Intelligence. My current background is in Electrical Engineering."],
    "orchestrator-test-session-04" 
)

In [ ]:
import os

# Get the path of the current working directory
agent_dir = os.getcwd()

print(f"Your Agent Directory (Agent Dir) is: \n{agent_dir}")